In [17]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [1]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [2]:
from nltk.corpus import stopwords
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
import numpy as np
import pandas as pd
import json
import os
import re

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Parameters
task = "classify_conds"
experiment_type = 1
numrows = 0
augment_type = "none"

batch_size = 16
learning_rate = 3e-5
weight_decay = 0.01
num_train_epochs = 1

In [6]:
# Filename
filename = "task-"+task+"_"+"exp-"+str(experiment_type)+"_"+"rows-"+str(numrows)+"_"+"agument-"+augment_type
filename

'task-classify_conds_exp-1_rows-0_agument-none'

In [5]:
# Save result
projpath = "/content/drive/MyDrive/UCB_MIDS/W266"

#saved_file = os.path.join(projpath, "saved_results",filename)

In [6]:
class tabulateResults:
  """ Tabulates results
  """
  def __init__(self, translate_file=None, agg_file=None, sel_file=None, conds_file=None):
    self.translate_file = translate_file
    self.agg_file = agg_file
    self.sel_file = sel_file
    self.conds_file = conds_file
    self.agg_exists = False
    self.sel_exists = False
    self.conds_exists = False
    self.dd = {"AGG": [""], "SEL": [""], "CONDS": [""], "EX": [""]}

    # read input files
    if self.translate_file != None:
      # set flags
      self.agg_exists, self.sel_exists, self.conds_exists = True, True, True
      # read json data
      self.trans_df = self.readfile(self.translate_file)
      # create AGG df
      self.agg_df = pd.DataFrame({})
      self.agg_df["pred_label"] = self.trans_df["pred_label"].apply(lambda x: self.extractAgg(x))
      self.agg_df["true_label"] = self.trans_df["true_label"].apply(lambda x: self.extractAgg(x))
      # create SEL df
      self.sel_df = pd.DataFrame({})
      self.sel_df["pred_label"] = self.trans_df["pred_label"].apply(lambda x: self.extractSel(x))
      self.sel_df["true_label"] = self.trans_df["true_label"].apply(lambda x: self.extractSel(x))
      # create CONDS df
      self.conds_df = pd.DataFrame({})
      self.conds_df["pred_label"] = self.trans_df["pred_label"].apply(lambda x: self.extractConds(x))
      self.conds_df["true_label"] = self.trans_df["true_label"].apply(lambda x: self.extractConds(x))
    else:
      if self.agg_file != None:
        self.agg_df = self.readfile(self.agg_file)
        self.agg_exists = True
      if self.sel_file != None:
        self.sel_df = self.readfile(self.sel_file)
        self.sel_exists = True
      if self.conds_file != None:
        self.conds_df = self.readfile(self.conds_file)
        self.conds_exists = True
    
    # AGG
    if self.agg_exists:
      # process each cond
      self.agg_df["true_label_proc"] = self.agg_df.true_label.apply(lambda x: self.processAggSel(x))
      self.agg_df["pred_label_proc"] = self.agg_df.pred_label.apply(lambda x: self.processAggSel(x))
      # evaluate match
      self.agg_df["match"] = self.agg_df.apply(lambda x: self.stringEqual(x["pred_label_proc"], x["true_label_proc"]), axis=1)
      # calculate accuracy
      self.dd["AGG"] = [round(100*self.agg_df["match"].mean(axis=0),1)]

    # SEL
    if self.sel_exists:
      # process each cond
      self.sel_df["true_label_proc"] = self.sel_df.true_label.apply(lambda x: self.processAggSel(x))
      self.sel_df["pred_label_proc"] = self.sel_df.pred_label.apply(lambda x: self.processAggSel(x))
      # evaluate match
      self.sel_df["match"] = self.sel_df.apply(lambda x: self.stringEqual(x["pred_label_proc"], x["true_label_proc"]), axis=1)
      # calculate accuracy
      self.dd["SEL"] = [round(100*self.sel_df["match"].mean(axis=0),1)]

    # CONDS
    if self.conds_exists:
      # process each cond
      self.conds_df["true_label_proc"] = self.conds_df.true_label.apply(lambda x: self.processConds(x))
      self.conds_df["pred_label_proc"] = self.conds_df.pred_label.apply(lambda x: self.processConds(x))
      # get condition count
      self.conds_df["true_numconds"] = self.conds_df.true_label_proc.apply(lambda x: self.countConds(x))
      self.conds_df["pred_numconds"] = self.conds_df.pred_label_proc.apply(lambda x: self.countConds(x))
      # evaluate match
      self.conds_df["match"] = self.conds_df.apply(lambda x: self.stringEqual(x["pred_label_proc"], x["true_label_proc"]), axis=1)
      # calculate accuracy    
      self.dd["CONDS"] = [round(100*self.conds_df["match"].mean(axis=0),1)]

    # calculate overall Execution accuracy
    if  self.agg_exists and self.sel_exists and self.conds_exists:
      # get each type's match
      a = np.array(self.agg_df["match"])
      s = np.array(self.sel_df["match"])
      c = np.array(self.conds_df["match"])
      # calculate overall EX match
      ex_match = a*s*c
      ex_match = round(100*ex_match.mean(),1)
      self.dd["EX"] = [ex_match]

    # generate final dataframe
    self.table = pd.DataFrame(self.dd)

  def readfile(self, file):
    """ Read input json file and take the second json element (the first one
        is only a header)
    """
    with open(file) as f:
      lines = f.readlines()
    count = 0
    for line in lines:
      d = json.loads(line.strip())
      if count == 0:
        pass
      else:
        df = pd.DataFrame(d)
      count += 1
    return df

  def extractAgg(self, txt):
    """ Extract AGG from pred and true labels of translate df 
    """
    pattern = r'\((.*?)\)'
    d = re.findall(pattern, txt)
    if d == []:
      agg = ""
    else:
      agg = d[0]
    return agg

  def extractSel(self, txt):
    """ Extract SEL from pred and true labels of translate df 
    """
    pattern = r'\[(.*?)\]'
    d = re.findall(pattern, txt)
    if d != []:
      sel = d[0]
    else:
      sel = ""
    return sel

  def extractConds(self, txt):
    """ Extract CONDS from pred and true labels of translate df 
    """
    pattern = r'\[(.*?)\]'
    d = re.findall(pattern, txt)
    conds = "".join(["["+n.replace("'", "")+"]" for n in d[2:]])
    return conds

  def processAggSel(self, txt):
    """ Strip, lowercase
    """
    return txt.strip().lower()

  def processConds(self, txt):
    """ Extract each where condition and post process it (strip, lowercase, sort for comparison).
    """
    pattern = r'\[(.*?)\]'
    extr_conds = re.findall(pattern, txt)
    out = [c.strip().lower() for c in extr_conds]
    out.sort()
    return out, len(out)

  def countConds(self, condlist):
    """ Count where conditions
    """
    return len(condlist) 
  
  def stringEqual(self, txt1, txt2):
    """ Check two strings or lists if they are equal
    """
    if txt1 == txt2:
      out = 1
    else:
      out = 0
    return out




### Direct Translate Result

In [7]:
task = "translate"

experiment_type = 0
numrows = 0
augment_type = "none"
filename = "task-"+task+"_"+"exp-"+str(experiment_type)+"_"+"rows-"+str(numrows)+"_"+"agument-"+augment_type
file = os.path.join(projpath, "saved_results",filename)
trans_exp0 = tabulateResults(translate_file=file)  

In [8]:
experiment_type = 1
numrows = 0
augment_type = "none"
filename = "task-"+task+"_"+"exp-"+str(experiment_type)+"_"+"rows-"+str(numrows)+"_"+"agument-"+augment_type
file = os.path.join(projpath, "saved_results",filename)
trans_exp1 = tabulateResults(translate_file=file)

In [9]:
experiment_type = 2
numrows = 0
augment_type = "none"
filename = "task-"+task+"_"+"exp-"+str(experiment_type)+"_"+"rows-"+str(numrows)+"_"+"agument-"+augment_type
file = os.path.join(projpath, "saved_results",filename)
trans_exp2 = tabulateResults(translate_file=file)

In [10]:
experiment_type = 1
numrows = 0
augment_type = "column"
filename = "task-"+task+"_"+"exp-"+str(experiment_type)+"_"+"rows-"+str(numrows)+"_"+"agument-"+augment_type
file = os.path.join(projpath, "saved_results",filename)
trans_exp1_aug = tabulateResults(translate_file=file)

In [11]:
experiment_type = 0
numrows = 0
augment_type = "column"
filename = "task-"+task+"_"+"exp-"+str(experiment_type)+"_"+"rows-"+str(numrows)+"_"+"agument-"+augment_type
file = os.path.join(projpath, "saved_results",filename)
trans_exp0_aug = tabulateResults(translate_file=file)

In [12]:
experiment_type = 0
numrows = 0
augment_type = "synonym"
filename = "task-"+task+"_"+"exp-"+str(experiment_type)+"_"+"rows-"+str(numrows)+"_"+"agument-"+augment_type+"1"
file = os.path.join(projpath, "saved_results",filename)
trans_exp0_aug_syn1 = tabulateResults(translate_file=file)

In [13]:
experiment_type = 0
numrows = 0
augment_type = "synonym"
filename = "task-"+task+"_"+"exp-"+str(experiment_type)+"_"+"rows-"+str(numrows)+"_"+"agument-"+augment_type+"2"
file = os.path.join(projpath, "saved_results",filename)
trans_exp0_aug_syn2 = tabulateResults(translate_file=file)

In [15]:
experiment_type = 3
numrows = 1
augment_type = "none"
filename = "task-"+task+"_"+"exp-"+str(experiment_type)+"_"+"rows-"+str(numrows)+"_"+"agument-"+augment_type
file = os.path.join(projpath, "saved_results",filename)
trans_exp3 = tabulateResults(translate_file=file)

In [27]:
experiment_type = 1
numrows = 0
augment_type = "synonym"
filename = "task-"+task+"_"+"exp-"+str(experiment_type)+"_"+"rows-"+str(numrows)+"_"+"agument-"+augment_type+"2"
file = os.path.join(projpath, "saved_results",filename)
trans_exp1_aug_syn2 = tabulateResults(translate_file=file)

In [17]:
trans_exp3.table

,AGG,SEL,CONDS,EX
0,86.8,92.3,73.3,62.3


In [28]:
finaltable_trans = pd.concat([trans_exp0.table, trans_exp1.table, trans_exp2.table, trans_exp3.table,
                              trans_exp1_aug.table, trans_exp0_aug.table, 
                              trans_exp0_aug_syn1.table, trans_exp0_aug_syn2.table, trans_exp1_aug_syn2.table])
finaltable_trans["Experiment"] = ["Standard", "+Schema", "+Schema ColTypes", "+Schema+ColTypes+Vals",
                                  "+Schema&Augmentation", "Augmentation", "Augmentation Synonym1", "Augmentation Synonym2",
                                  "+Schema&Augmentation Synonym2"]
finaltable_trans = finaltable_trans.set_index('Experiment')
finaltable_trans.style.set_properties(**{'text-align': 'left'})

,AGG,SEL,CONDS,EX
Experiment,,,,
Standard,84.200000,65.700000,46.000000,32.000000
+Schema,86.900000,93.300000,74.100000,63.100000
+Schema ColTypes,87.100000,92.900000,73.100000,62.200000
+Schema+ColTypes+Vals,86.800000,92.300000,73.300000,62.300000
+Schema&Augmentation,87.600000,93.800000,78.300000,66.800000
Augmentation,84.600000,69.000000,52.300000,37.600000
Augmentation Synonym1,84.100000,61.500000,43.700000,27.300000
Augmentation Synonym2,83.300000,57.200000,40.300000,25.500000
+Schema&Augmentation Synonym2,86.200000,90.300000,68.300000,56.300000


In [69]:
# write csv
csv_file = os.path.join(projpath, "saved_results", "finaltable_trans.csv")
finaltable_trans.to_csv(csv_file)


### Slot Prediction Result

```
# This is formatted as code
```



In [15]:
tasks = ["classify_agg", "classify_sel", "classify_conds"]

In [16]:
experiment_type = 0
numrows = 0
augment_type = "none"

def genfilename(tasks, experiment_type, numrows=0, augment_type="none"):
  files = []
  for task in tasks:
    filename = "task-"+task+"_"+"exp-"+str(experiment_type)+"_"+"rows-"+str(numrows)+"_"+"agument-"+augment_type
    name = os.path.join(projpath, "saved_results",filename)
    files.append(name)
  return files

files = genfilename(tasks, experiment_type, numrows, augment_type)
exp0 = tabulateResults(agg_file=files[0], sel_file=files[1], conds_file=files[2])


In [17]:
experiment_type = 1
numrows = 0
augment_type = "none"

def genfilename(tasks, experiment_type, numrows=0, augment_type="none"):
  files = []
  for task in tasks:
    filename = "task-"+task+"_"+"exp-"+str(experiment_type)+"_"+"rows-"+str(numrows)+"_"+"agument-"+augment_type
    name = os.path.join(projpath, "saved_results",filename)
    files.append(name)
  return files

files = genfilename(tasks, experiment_type, numrows, augment_type)
exp1 = tabulateResults(agg_file=files[0], sel_file=files[1], conds_file=files[2])


In [18]:
experiment_type = 1
numrows = 0
augment_type = "column"

def genfilename(tasks, experiment_type, numrows=0, augment_type="none"):
  files = []
  for task in tasks:
    filename = "task-"+task+"_"+"exp-"+str(experiment_type)+"_"+"rows-"+str(numrows)+"_"+"agument-"+augment_type
    name = os.path.join(projpath, "saved_results",filename)
    files.append(name)
  return files

files = genfilename(tasks, experiment_type, numrows, augment_type)
exp1_aug = tabulateResults(agg_file=files[0], sel_file=files[1], conds_file=files[2])

In [40]:
experiment_type = 1
numrows = 0
augment_type = "synonym"

def genfilename(tasks, experiment_type, numrows=0, augment_type="none"):
  files = []
  for task in tasks:
    filename = "task-"+task+"_"+"exp-"+str(experiment_type)+"_"+"rows-"+str(numrows)+"_"+"agument-"+augment_type
    name = os.path.join(projpath, "saved_results",filename)
    files.append(name)
  return files

files = genfilename(tasks, experiment_type, numrows, augment_type)
exp1_aug_syn = tabulateResults(agg_file=None, sel_file=None, conds_file=files[2])

In [42]:
finaltable = pd.concat([exp0.table, exp1.table, exp1_aug.table, exp1_aug_syn.table])
finaltable["Experiment"] = ["Standard", "+Schema", "+Schema&AugColumn", "+Schema&AugSynonym"]
finaltable = finaltable.set_index('Experiment')
finaltable.style.set_properties(**{'text-align': 'left'})

,AGG,SEL,CONDS,EX
Experiment,,,,
Standard,88.900000,72.000000,45.400000,33.200000
+Schema,89.400000,92.100000,71.800000,60.500000
+Schema&AugColumn,90.000000,93.600000,76.200000,65.300000
+Schema&AugSynonym,,,65.900000,


In [288]:
# write csv
csv_file = os.path.join(projpath, "saved_results", "finaltable.csv")
finaltable.to_csv(csv_file)


### Effect of each experiment over Plain task

In [21]:
finaltable_trans

,AGG,SEL,CONDS,EX
Experiment,,,,
Standard,84.2,65.7,46.0,32.0
+Schema,86.9,93.3,74.1,63.1
+Schema ColTypes,87.1,92.9,73.1,62.2
+Schema+ColTypes+Vals,86.8,92.3,73.3,62.3
+Schema&Augmentation,87.6,93.8,78.3,66.8
Augmentation,84.6,69.0,52.3,37.6
Augmentation Synonym1,84.1,61.5,43.7,27.3
Augmentation Synonym2,83.3,57.2,40.3,25.5


In [22]:
# schema (col names)
schema = np.around(np.array(finaltable_trans.loc[["+Schema"],:]) - np.array(finaltable_trans.loc[["Standard"],:]), decimals=1)
# schema (col names + types)
schema_ct = np.around(np.array(finaltable_trans.loc[["+Schema ColTypes"],:]) - np.array(finaltable_trans.loc[["Standard"],:]), decimals=1)
# schema (col names + types + vals)
schema_vals = np.around(np.array(finaltable_trans.loc[["+Schema+ColTypes+Vals"],:]) - np.array(finaltable_trans.loc[["Standard"],:]), decimals=1)
# augmentation (sel col)
aug_column = np.around(np.array(finaltable_trans.loc[["Augmentation"],:]) - np.array(finaltable_trans.loc[["Standard"],:]), decimals=1)
# augmentation (1 synonym)
aug_column_syn1 = np.around(np.array(finaltable_trans.loc[["Augmentation Synonym1"],:]) - np.array(finaltable_trans.loc[["Standard"],:]), decimals=1)
# augmentation (2 synonyms)
aug_column_syn2 = np.around(np.array(finaltable_trans.loc[["Augmentation Synonym2"],:]) - np.array(finaltable_trans.loc[["Standard"],:]), decimals=1)

In [25]:
array = np.stack((schema[0], schema_ct[0], schema_vals[0], aug_column[0], aug_column_syn1[0], aug_column_syn2[0]), axis = 0)
index_values = ['Schema: Column', 'Schema: Column + Types', 'Schema: Column + Types + Vals', 'Augmentation: Select Column',
                'Augmentation: Synonym - 1 word', 'Augmentation: Synonym - 2 words']
column_values = ['Agg', 'Sel', 'Conds', 'Ex']
exp_impacts = pd.DataFrame(data = array, 
                  index = index_values, 
                  columns = column_values)

In [26]:
exp_impacts

,Agg,Sel,Conds,Ex
Schema: Column,2.7,27.6,28.1,31.1
Schema: Column + Types,2.9,27.2,27.1,30.2
Schema: Column + Types + Vals,2.6,26.6,27.3,30.3
Augmentation: Select Column,0.4,3.3,6.3,5.6
Augmentation: Synonym - 1 word,-0.1,-4.2,-2.3,-4.7
Augmentation: Synonym - 2 words,-0.9,-8.5,-5.7,-6.5


In [27]:
# write csv
csv_file = os.path.join(projpath, "saved_results", "exp_impacts.csv")
exp_impacts.to_csv(csv_file)

In [91]:
schema[0]

array([ 2.7, 27.6, 28.1, 31.1])

In [163]:
task = "translate"
experiment_type = 0
numrows = 0
augment_type = "none"
filename = "task-"+task+"_"+"exp-"+str(experiment_type)+"_"+"rows-"+str(numrows)+"_"+"agument-"+augment_type
file = os.path.join(projpath, "saved_results",filename)
file

'/content/drive/MyDrive/UCB_MIDS/W266/saved_results/task-translate_exp-0_rows-0_agument-none'

In [164]:
# read file
with open(file) as f:
  lines = f.readlines()

count = 0
for line in lines:
    d = json.loads(line.strip())
    if count == 0:
      pass
    else:
      df = pd.DataFrame(d)

    count += 1

In [169]:
df.tail()

,pred_label,true_label
15873,select (count) [points] from [2-1686868-1] whe...,select [points] from [2-1226335-1] where [yea...
15874,select [chassis] from [2-1686868-1] where [poi...,select [chassis] from [2-1226335-1] where [po...
15875,select (count) [points] from [2-1686868-1] whe...,select [points] from [2-1226335-1] where [eng...
15876,select [chassis] from [2-1686868-1] where [yea...,select [chassis] from [2-1226335-1] where [en...
15877,select [year] from [2-1686868-1] where [points...,select (sum) [year] from [2-1226335-1] where [...


In [86]:
txt = df.iloc[15876,:]["true_label"]
pattern = r'\[(.*?)\]'
extr_conds = re.findall(pattern, txt)
extr_conds

['engine equals to ford v8', 'year greater than 1976', 'points equals to 12']

In [87]:
txt = df.iloc[15876,:]["pred_label"]
pattern = r'\[(.*?)\]'
extr_conds = re.findall(pattern, txt)
extr_conds

['year greater than 1976', 'points equals to 12']

In [265]:
#txt = "select [salary] FROM [table_1] where [company equals to apple] and [location equals to san jose]"
txt = "select 5 imports of plywood"
pattern = r'\[(.*?)\]'
re.findall(pattern, txt)

[]

In [200]:
pattern = r'\((.*?)\)'
re.findall(pattern, txt)

[]

### https://maelfabien.github.io/machinelearning/NLP_8/#synonym-replacement-sr

In [4]:
from nltk.corpus import wordnet 

def get_synonyms(word):
    """
    Get synonyms of a word
    """
    synonyms = set()
    
    for syn in wordnet.synsets(word): 
        for l in syn.lemmas(): 
            synonym = l.name().replace("_", " ").replace("-", " ").lower()
            synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
            synonyms.add(synonym) 
    
    if word in synonyms:
        synonyms.remove(word)
    
    return list(synonyms)

In [15]:
import random

def synonym_replacement(words, n):
    
    stop_words = list(set(stopwords.words('english')))
    words = words.split()
    
    new_words = words.copy()
    random_word_list = list(set([word for word in words if word not in stop_words]))
    random.shuffle(random_word_list)
    num_replaced = 0
    
    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        
        if len(synonyms) >= 1:
            synonym = random.choice(list(synonyms))
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        
        if num_replaced >= n: #only replace up to n words
            break

    sentence = ' '.join(new_words)

    return sentence

In [149]:
txt = "how much does an engineer make on average?" 
n = 2
synonym_replacement(txt, n)

'how often does an engineer spend a penny on average?'

In [150]:
min(2,5)

2